In [213]:
from deep_learning_features_audio import *
from deep_learning_dict_api import AudioAnalysisAPI
from deep_learning_dict_datasets import Datasets
import evaluate
from evaluate import load
import pandas as pd
from pathlib import Path
import torch
from torchmetrics import Accuracy
from datetime import datetime
import json


In [214]:
def emotion_recognition_evaluate_metric_with_model_on_IEMOCAP(task, dataset, model, metrics, n_test):
    test_done = 0
    errors = 0
    predictions = []
    references = []
    result = {
        "evaluation": {}
    }

    emotions_dict ={
        "ang":0,
        "dis":1, 
        "exc":2, 
        "fea":3,
        "fru":4,
        "hap":5,
        "neu":6,
        "oth":7,	
        "sad":8,
        "sur":9,
        "xxx":10,
    }
    # os.walk(dataset_path)
    test_table = pd.read_csv(Datasets[task][dataset]["path"])
    tot_sample = test_table.shape[0]
    if n_test is None:
        n_test = tot_sample
    for i, row in enumerate(test_table.iterrows()):
        print("Benchmarking: {}/{}".format(i, test_table.shape[0]))
        # print("row 0 :{}".format(row[0]))
        # print("row 1 :{}".format(row[1]["audio_path"]))

        audio_path = row[1]["audio_path"]
        reference = row[1]["emotion_evaluation_1"]
        # print("audio_path:{}".format(audio_path))
        # print("reference:{}".format(reference))
        if i == n_test:
                break
        try:
            if os.path.isfile(audio_path):
                prediction = AudioAnalysisAPI[model]['function'](audiofile_path=audio_path)
                if prediction is not None and prediction != "":
                    emotion_index_prediction = emotions_dict[prediction[0]]
                    emotion_index_reference = emotions_dict[reference]
                    predictions.append(emotion_index_prediction)
                    references.append(emotion_index_reference)
                    print("audiofile_path: {}".format(audio_path))
                    print("reference: {}, index:{}".format(reference, emotion_index_reference))
                    print("prediction:{}, index:{}".format(prediction[0], emotion_index_prediction))
                    print()
                    test_done += 1
            else:
                print(audio_path, " file doesn't exist")
                errors+=1
            
        except Exception as e:
            print(e)
            errors += 1
            pass

    target = torch.tensor(references)
    preds = torch.tensor(predictions)
    accuracy = Accuracy()
    accuracy_result = accuracy(preds, target)
    print("==> accuracy_result:{}".format(accuracy_result))
    # Creating the content of result file
    result ={
        "model": model, 
        "dataset": dataset, 
        "tot_sample":tot_sample,
        "n_test": n_test,
        "n_test_done": test_done,
        "accuracy":str(accuracy_result),
    }

    # Creating filename
    dateTimeObj = datetime.now()
    print(dateTimeObj)
    timestampStr = dateTimeObj.strftime("%d_%b_%Y__%H_%M_%S_%f")
    result_filename = timestampStr + "_evaluate_" + model.split("/")[-1] + "_" + dataset + "_" + str(n_test)+".json"
    print('result_filename : ', result_filename)
    with open(result_filename, 'w') as f:
        json.dump(result, f)
    
    return result    
    


In [215]:
task = "Emotion Recognition"
dataset = "IEMOCAP"
models = [
    '/api/emotion_recognition/wav2vec2_IEMOCAP',
    ]
metrics = ["accuracy"]

n_test = 10

for model in models:
    print("===== Benchmark of model: {} -- dataset: {} ".format(model.split("/")[-1], dataset))
    emotion_recognition_evaluate_metric_with_model_on_IEMOCAP(
        task=task,
        dataset=dataset,
        model=model,
        metrics=metrics,
        n_test=n_test
    )

===== Benchmark of model: wav2vec2_IEMOCAP -- dataset: IEMOCAP 
Benchmarking: 0/10039
row 0 :0
row 1 :/storage/data_8T/datasets/audio/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script01_1/Ses01M_script01_1_F000.wav
audio_path:/storage/data_8T/datasets/audio/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script01_1/Ses01M_script01_1_F000.wav
reference:fru


/home/valeriopuglisi/.conda/envs/DLAABE/lib/python3.10/site-packages/transformers/configuration_utils.py:363: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2Model: ['quantizer.weight_proj.bias', 'project_q.bias', 'quantizer.weight_proj.weight', 'quantizer.codevectors', 'project_q.weight', 'project_hid.weight', 'project_hid.bias']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoi

audiofile_path: /storage/data_8T/datasets/audio/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script01_1/Ses01M_script01_1_F000.wav
reference: fru, index:4
prediction:neu, index:6

Benchmarking: 1/10039
row 0 :1
row 1 :/storage/data_8T/datasets/audio/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script01_1/Ses01M_script01_1_F001.wav
audio_path:/storage/data_8T/datasets/audio/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script01_1/Ses01M_script01_1_F001.wav
reference:sur


Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2Model: ['quantizer.weight_proj.bias', 'project_q.bias', 'quantizer.weight_proj.weight', 'quantizer.codevectors', 'project_q.weight', 'project_hid.weight', 'project_hid.bias']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
speechbrain.lobes.models.huggingface_wav2vec - wav2vec 2.0 is frozen.


audiofile_path: /storage/data_8T/datasets/audio/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script01_1/Ses01M_script01_1_F001.wav
reference: sur, index:9
prediction:ang, index:0

Benchmarking: 2/10039
row 0 :2
row 1 :/storage/data_8T/datasets/audio/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script01_1/Ses01M_script01_1_F002.wav
audio_path:/storage/data_8T/datasets/audio/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script01_1/Ses01M_script01_1_F002.wav
reference:sur


...

audiofile_path: /storage/data_8T/datasets/audio/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script01_1/Ses01M_script01_1_F007.wav
reference: fru, index:4
prediction:ang, index:0

Benchmarking: 8/10039
row 0 :8
row 1 :/storage/data_8T/datasets/audio/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script01_1/Ses01M_script01_1_F008.wav
audio_path:/storage/data_8T/datasets/audio/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script01_1/Ses01M_script01_1_F008.wav
reference:ang


Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2Model: ['quantizer.weight_proj.bias', 'project_q.bias', 'quantizer.weight_proj.weight', 'quantizer.codevectors', 'project_q.weight', 'project_hid.weight', 'project_hid.bias']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
speechbrain.lobes.models.huggingface_wav2vec - wav2vec 2.0 is frozen.


audiofile_path: /storage/data_8T/datasets/audio/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script01_1/Ses01M_script01_1_F008.wav
reference: ang, index:0
prediction:ang, index:0

Benchmarking: 9/10039
row 0 :9
row 1 :/storage/data_8T/datasets/audio/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script01_1/Ses01M_script01_1_F009.wav
audio_path:/storage/data_8T/datasets/audio/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script01_1/Ses01M_script01_1_F009.wav
reference:xxx


Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2Model: ['quantizer.weight_proj.bias', 'project_q.bias', 'quantizer.weight_proj.weight', 'quantizer.codevectors', 'project_q.weight', 'project_hid.weight', 'project_hid.bias']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
speechbrain.lobes.models.huggingface_wav2vec - wav2vec 2.0 is frozen.


audiofile_path: /storage/data_8T/datasets/audio/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script01_1/Ses01M_script01_1_F009.wav
reference: xxx, index:10
prediction:neu, index:6

Benchmarking: 10/10039
row 0 :10
row 1 :/storage/data_8T/datasets/audio/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script01_1/Ses01M_script01_1_F010.wav
audio_path:/storage/data_8T/datasets/audio/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script01_1/Ses01M_script01_1_F010.wav
reference:sur
==> accuracy_result:0.20000000298023224
2022-10-14 14:55:58.119538
result_filename :  14_Oct_2022__14_55_58_119538_evaluate_wav2vec2_IEMOCAP_IEMOCAP_10.json
